# Лаб 2. Матричные разложения

## Теория

Суть методов, основанных на матричном разложении состоит в том, что каждому пользователю $u$ ставится в соответствие вектор интересов $p_u$ длины $k$, а каждому айтему $i$ ставится в соответствие вектор интересов $q_i$, которым он удовлетворяет. Таким образом, что скалярное произведение этих векторов даёт предсказание $\hat r_{ui}$:

$$
\hat r_{ui} = p_u \dot q_i
$$

Такие вектора $p_u$ и $q_i$ составляют матрицы $P$ и $Q$ соответственно, которые в произведении даёт матрицу предсказаний $\hat R$.

![Разложение матриц](./images/PQ.drawio.png)

## Код

Ставим библиотеки

In [1]:
#%pip install --quiet -U scikit-surprise

Импорты

In [121]:
from tqdm.notebook import tqdm
import pandas as pd

# Реализации методов матричного разложения будем использовать из библиотеки surprise
from surprise import Dataset, SVD, NMF, Reader
# Кроссвалидацию проходили на машинном обучении
from surprise.model_selection import cross_validate
# Библиотека для визуализации прогресса в питоновском ноутбуке
from tqdm.notebook import tqdm

import numpy as np

Загрузка датасета (из первой лабораторной)

In [250]:
data = Dataset.load_from_file('./ml-latest-small/ratings.csv', Reader(sep=',', line_format="user item rating timestamp", skip_lines=1))
trainset = data.build_full_trainset()

user_id = '42'
item_id = '50'

In [85]:
trainset.ur

defaultdict(list,
            {0: [(0, 4.0),
              (1, 4.0),
              (2, 4.0),
              (3, 5.0),
              (4, 5.0),
              (5, 3.0),
              (6, 5.0),
              (7, 4.0),
              (8, 5.0),
              (9, 5.0),
              (10, 5.0),
              (11, 5.0),
              (12, 3.0),
              (13, 5.0),
              (14, 4.0),
              (15, 5.0),
              (16, 3.0),
              (17, 3.0),
              (18, 5.0),
              (19, 4.0),
              (20, 4.0),
              (21, 5.0),
              (22, 4.0),
              (23, 3.0),
              (24, 4.0),
              (25, 5.0),
              (26, 4.0),
              (27, 3.0),
              (28, 5.0),
              (29, 4.0),
              (30, 4.0),
              (31, 5.0),
              (32, 4.0),
              (33, 4.0),
              (34, 4.0),
              (35, 5.0),
              (36, 5.0),
              (37, 3.0),
              (38, 5.0),
              

### SVD

In [5]:
# Обучаем модель
algo_svd = SVD(biased=False, n_factors=100, random_state=0)
algo_svd = algo_svd.fit(trainset)

In [6]:
# Даём предсказание для юзера и произвольного айтема
algo_svd.predict(user_id, item_id).est

4.843205752439971

Матричное представление пользователей

In [7]:
algo_svd.pu.shape

(610, 100)

In [8]:
algo_svd.pu

array([[-6.87150123e-02, -4.97125466e-01,  5.22490481e-01, ...,
         3.00384957e-01, -2.75593242e-02,  1.99262616e-01],
       [-1.40247982e-02, -5.49300486e-01,  2.15699484e-01, ...,
         2.86632122e-01,  3.70989216e-01,  3.19732723e-01],
       [-1.39276531e-01, -9.70704643e-02,  1.96391539e-01, ...,
         1.94101607e-01,  1.05463265e-01, -1.48543866e-01],
       ...,
       [-2.06136144e-01, -3.40536072e-01,  1.09172410e-01, ...,
         1.40281605e-01,  4.91498759e-01,  2.21511770e-01],
       [ 3.70795739e-02, -3.13546780e-03, -1.90401994e-02, ...,
         1.01903484e-01, -2.82033528e-02,  2.27518375e-01],
       [-2.59993731e-01, -3.40010283e-03,  3.14725603e-01, ...,
        -4.86764605e-04,  3.01699954e-01, -3.37481190e-01]])

Матричное представление товаров

In [9]:
algo_svd.qi.shape

(9724, 100)

In [10]:
algo_svd.qi

array([[ 0.2043476 , -0.20209592,  0.01270568, ..., -0.15501468,
        -0.19630591,  0.13715467],
       [-0.06753726,  0.07357937, -0.10516829, ..., -0.03180826,
        -0.16027366,  0.06335369],
       [-0.10061689, -0.24027688,  0.03855881, ...,  0.10082811,
         0.16416403,  0.01194065],
       ...,
       [-0.09226767,  0.21245959,  0.05063394, ..., -0.01182451,
         0.16429169, -0.02939865],
       [-0.09816132, -0.02186426,  0.1754864 , ..., -0.08722964,
         0.11996007, -0.27831176],
       [-0.19709193, -0.0855968 ,  0.0375563 , ...,  0.04910192,
         0.00400957,  0.05660991]])

Пример связи матричных представлений и итогово предсказания

In [11]:
svd_item_id = algo_svd.trainset.to_inner_iid(item_id)
svd_user_id = algo_svd.trainset.to_inner_uid(user_id)

# Умножаем вектор интересов пользователя на вектор интересов айтема
(algo_svd.pu[svd_user_id]*algo_svd.qi[svd_item_id]).sum()

4.84320575243997

### Non-negative Matrix Factorization

По интерфейсу и внутреннему устройству этот метод похож на SVD. Повторите предыдущие шаги для него.

In [12]:
# Обучаем модель
algo_nmf = NMF(n_factors=15, random_state=0)
algo_nmf = algo_nmf.fit(trainset)

In [13]:
# Даём предсказание для юзера и произвольного айтема
algo_nmf.predict(user_id, item_id).est

4.421082909663385

In [14]:
algo_nmf.pu.shape

(610, 15)

In [15]:
algo_nmf.qi.shape

(9724, 15)

In [16]:
# Пример связи матричных представлений и итогово предсказания
nmf_item_id = algo_nmf.trainset.to_inner_iid(item_id)
nmf_user_id = algo_nmf.trainset.to_inner_uid(user_id)

# Умножаем вектор интересов пользователя на вектор интересов айтема
(algo_nmf.pu[nmf_user_id]*algo_nmf.qi[nmf_item_id]).sum()

4.421082909663385

### Валидация и обучение
Используя кроссвалидацию, сравните представленные алгоритмы по качеству.

Подберите гиперпараметры моделей для улучшения результатов.

Расшифровка метрик ошибок и альтернативы можно найти в документации (как и параметры моделей): https://surprise.readthedocs.io/en/stable/accuracy.html#module-surprise.accuracy

In [17]:
cross_validate(algo=algo_svd, data=data, measures=["mse", "rmse", "mae"], cv=5, verbose=True)

Evaluating MSE, RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MSE (testset)     0.9532  0.9580  0.9625  0.9531  0.9573  0.9568  0.0035  
RMSE (testset)    0.9763  0.9787  0.9811  0.9763  0.9784  0.9782  0.0018  
MAE (testset)     0.7527  0.7485  0.7567  0.7497  0.7544  0.7524  0.0030  
Fit time          0.84    0.73    0.72    0.69    0.71    0.74    0.05    
Test time         0.13    0.08    0.08    0.12    0.12    0.11    0.02    


{'test_mse': array([0.95320998, 0.95795089, 0.96249107, 0.95313435, 0.95726595]),
 'test_rmse': array([0.97632473, 0.97874965, 0.98106629, 0.976286  , 0.97839969]),
 'test_mae': array([0.75273667, 0.74853895, 0.75669152, 0.74971539, 0.75442978]),
 'fit_time': (0.8401896953582764,
  0.7267739772796631,
  0.7161829471588135,
  0.6919422149658203,
  0.7050056457519531),
 'test_time': (0.12990140914916992,
  0.08274149894714355,
  0.0777890682220459,
  0.11768484115600586,
  0.11868476867675781)}

In [18]:
cross_validate(algo=algo_nmf, data=data, measures=["mse", "rmse", "mae"], cv=5, verbose=True)

Evaluating MSE, RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MSE (testset)     0.8445  0.8520  0.8571  0.8436  0.8560  0.8506  0.0057  
RMSE (testset)    0.9190  0.9230  0.9258  0.9185  0.9252  0.9223  0.0031  
MAE (testset)     0.7032  0.7073  0.7075  0.7064  0.7114  0.7072  0.0026  
Fit time          1.31    1.27    1.22    1.23    1.26    1.26    0.03    
Test time         0.09    0.08    0.12    0.13    0.07    0.10    0.02    


{'test_mse': array([0.84448436, 0.85196499, 0.85708404, 0.84356111, 0.8560095 ]),
 'test_rmse': array([0.9189583 , 0.92301949, 0.92578834, 0.91845583, 0.92520782]),
 'test_mae': array([0.70322865, 0.70731507, 0.70752841, 0.70635339, 0.7113575 ]),
 'fit_time': (1.3097748756408691,
  1.271446704864502,
  1.2212896347045898,
  1.2287898063659668,
  1.255704402923584),
 'test_time': (0.08776402473449707,
  0.07579708099365234,
  0.1246638298034668,
  0.12566542625427246,
  0.07184290885925293)}

In [19]:
# # Подбор лучшего n_factors для SVD -> 5
# svd_rmse = {}

# for i in tqdm(range(1, 21)):
#     algo_svd = SVD(biased=False, n_factors=i, random_state=0)
#     algo_svd = algo_svd.fit(trainset)
#     svd_rmse[i]=cross_validate(algo_svd, data, verbose=False)['test_rmse'].mean()
# {k: v for k, v in sorted(svd_rmse.items(), key=lambda item: item[1])}

In [20]:
# # Подбор лучшего n_epochs для SVD -> 58
# svd_epochs = {}

# for i in tqdm(range(1, 100)):
#     algo_svd = SVD(biased=False, n_factors=5, n_epochs=i, random_state=0)
#     algo_svd = algo_svd.fit(trainset)
#     svd_epochs[i]=cross_validate(algo_svd, data, verbose=False)['test_rmse'].mean()
# {k: v for k, v in sorted(svd_epochs.items(), key=lambda item: item[1])}

In [21]:
# # Подбор лучшего n_factors для NMF -> 18
# nmf_rmse={}

# for i in tqdm(range(1, 21)):
#     algo_nmf = NMF(biased=False, n_factors=i, random_state=0)
#     algo_nmf = algo_nmf.fit(trainset)
#     nmf_rmse[i]=cross_validate(algo_nmf, data, verbose=False)['test_rmse'].mean()
# {k: v for k, v in sorted(nmf_rmse.items(), key=lambda item: item[1])}

In [22]:
# # Подбор лучшего n_epochs для NMF -> 74
# nmf_epochs={}

# for i in tqdm(range(1, 100)):
#     algo_nmf = NMF(biased=False, n_factors=18, n_epochs=i, random_state=0)
#     algo_nmf = algo_nmf.fit(trainset)
#     nmf_epochs[i]=cross_validate(algo_nmf, data, verbose=False)['test_rmse'].mean()
# {k: v for k, v in sorted(nmf_epochs.items(), key=lambda item: item[1])}

In [23]:
algo_svd = SVD(biased=False, n_factors=5, n_epochs=58, random_state=0)
algo_svd = algo_svd.fit(trainset)
cross_validate(algo=algo_svd, data=data, measures=["mse", "rmse", "mae"], cv=5, verbose=True)

Evaluating MSE, RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MSE (testset)     0.8227  0.8200  0.8251  0.8078  0.7995  0.8150  0.0098  
RMSE (testset)    0.9071  0.9056  0.9083  0.8988  0.8942  0.9028  0.0054  
MAE (testset)     0.6926  0.6915  0.6944  0.6875  0.6856  0.6903  0.0033  
Fit time          1.05    0.96    1.24    0.96    0.83    1.01    0.14    
Test time         0.14    0.15    0.08    0.14    0.14    0.13    0.02    


{'test_mse': array([0.82274101, 0.82002978, 0.82506415, 0.80778812, 0.7995116 ]),
 'test_rmse': array([0.90705072, 0.90555496, 0.90833042, 0.89877033, 0.89415413]),
 'test_mae': array([0.69259579, 0.69147399, 0.69438208, 0.68747016, 0.68562843]),
 'fit_time': (1.0524084568023682,
  0.9640347957611084,
  1.2409143447875977,
  0.9557242393493652,
  0.8258235454559326),
 'test_time': (0.1421494483947754,
  0.15259218215942383,
  0.0820469856262207,
  0.13962531089782715,
  0.1356372833251953)}

In [24]:
algo_nmf = NMF(biased=False, n_factors=18, n_epochs=74, random_state=0)
algo_nmf = algo_nmf.fit(trainset)
cross_validate(algo=algo_nmf, data=data, measures=["mse", "rmse", "mae"], cv=5, verbose=True)

Evaluating MSE, RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MSE (testset)     0.8473  0.8348  0.8577  0.8366  0.8536  0.8460  0.0090  
RMSE (testset)    0.9205  0.9137  0.9261  0.9147  0.9239  0.9198  0.0049  
MAE (testset)     0.7033  0.7008  0.7112  0.7029  0.7073  0.7051  0.0037  
Fit time          2.41    2.40    2.06    1.91    2.14    2.18    0.20    
Test time         0.08    0.13    0.13    0.07    0.21    0.12    0.05    


{'test_mse': array([0.84729482, 0.83484326, 0.85769334, 0.83663359, 0.85357354]),
 'test_rmse': array([0.92048619, 0.91369758, 0.92611735, 0.91467677, 0.92389044]),
 'test_mae': array([0.70325639, 0.70077918, 0.71121447, 0.7029378 , 0.70734064]),
 'fit_time': (2.4093477725982666,
  2.4032301902770996,
  2.063220500946045,
  1.9092717170715332,
  2.1375370025634766),
 'test_time': (0.08078384399414062,
  0.12666082382202148,
  0.12566065788269043,
  0.07280611991882324,
  0.2051405906677246)}

## Результат
                                                                  
| SVD      | Было     | Стало    |
|----------|----------|----------|
| MSE      | 0.9510   | 0.8157   |
| RMSE     | 0.9752   | 0.9032   |
| MAE      | 0.7522   | 0.6909   |

| NnMF     | Было     | Стало    |
|----------|----------|----------|
| MSE      | 0.8529   | 0.8425   |
| RMSE     | 0.9235   | 0.9179   |
| MAE      | 0.7072   | 0.7042   |
      

## Призрак в машине

Проинтерпретируйте получившиеся "интересы" в модели NnMF.

Например, можно начать с просмотра отсортированных по ним фильмов

In [25]:
movies = pd.read_csv('./ml-latest-small/movies.csv', delimiter=',')
# Делаем таблицу для преобразования id в имя
id_to_title = movies.loc[:, ["movieId", "title"]]
id_to_title.set_index("movieId", inplace=True)

In [26]:
factor_idx = 17
items = list(algo_nmf.trainset.all_items())

items.sort(key=lambda inner_id: -algo_nmf.qi[inner_id][factor_idx])
named_items = [[id_to_title.loc[int(trainset.to_raw_iid(k))]["title"], [int(f*100) for f in algo_nmf.qi[k]]] for k in items]
named_items

[['Herbie: Fully Loaded (2005)',
  [55, 27, 30, 12, 0, 4, 60, 4, 52, 48, 6, 75, 5, 6, 23, 75, 2, 189]],
 ['Bring Me the Head of Alfredo Garcia (1974)',
  [5, 54, 23, 58, 2, 50, 153, 3, 44, 7, 50, 1, 25, 4, 2, 25, 2, 186]],
 ['Outsourced (2006)',
  [76, 78, 0, 0, 23, 3, 11, 1, 66, 0, 0, 0, 7, 0, 67, 43, 7, 175]],
 ['American Dreamz (2006)',
  [45, 6, 1, 44, 2, 0, 186, 0, 68, 0, 24, 1, 0, 0, 0, 0, 31, 168]],
 ['Rescue Dawn (2006)',
  [19, 53, 64, 95, 15, 78, 73, 9, 77, 37, 31, 3, 10, 41, 0, 19, 0, 167]],
 ['Pride and Prejudice and Zombies (2016)',
  [112, 0, 0, 0, 2, 103, 0, 17, 3, 0, 7, 40, 181, 14, 62, 2, 0, 166]],
 ['Grave of the Fireflies (Hotaru no haka) (1988)',
  [18, 79, 60, 92, 5, 24, 38, 8, 3, 45, 1, 1, 72, 21, 0, 12, 60, 163]],
 ['Double, The (2011)',
  [23, 60, 46, 75, 26, 71, 121, 24, 83, 21, 66, 39, 31, 28, 6, 3, 30, 163]],
 ['Yogi Bear (2010)',
  [17, 21, 126, 19, 27, 17, 0, 0, 28, 11, 136, 46, 2, 0, 75, 0, 0, 162]],
 ['Pride and Prejudice (1995)',
  [0, 39, 36, 67, 59, 65

In [27]:
frequency_of_genres_start = {}
for i in range (1000):
    genres = movies.loc[(movies['title'] == named_items[i][0])]['genres'].reset_index(drop=True)[0].split("|")
    for j in genres:
        if j in frequency_of_genres_start:
            frequency_of_genres_start[j] = frequency_of_genres_start[j] + 1
        else:
            frequency_of_genres_start[j] = 1
    #print(named_items[i][0],'\t\t',movies.loc[(movies['title'] == named_items[i][0])]['genres'].reset_index(drop=True)[0])
frequency_of_genres_start = {k: v for k, v in sorted(frequency_of_genres_start.items(), key=lambda item: item[1], reverse=True)}
print(frequency_of_genres_start)
    
print("-----------")
frequency_of_genres_end = {}
for i in range (1000,0,-1):
    genres = movies.loc[(movies['title'] == named_items[-i][0])]['genres'].reset_index(drop=True)[0].split("|")
    for j in genres:
        if j in frequency_of_genres_end:
            frequency_of_genres_end[j] = frequency_of_genres_end[j] + 1
        else:
            frequency_of_genres_end[j] = 1
    #print(named_items[-i][0],'\t\t',movies.loc[(movies['title'] == named_items[-i][0])]['genres'].reset_index(drop=True)[0]) 
frequency_of_genres_end = {k: v for k, v in sorted(frequency_of_genres_end.items(), key=lambda item: item[1], reverse=True)}
print(frequency_of_genres_end)
 

{'Drama': 453, 'Comedy': 381, 'Action': 204, 'Thriller': 197, 'Adventure': 148, 'Romance': 147, 'Crime': 128, 'Sci-Fi': 99, 'Horror': 92, 'Fantasy': 77, 'Children': 71, 'Animation': 63, 'Mystery': 63, 'Documentary': 45, 'War': 39, 'Musical': 38, 'IMAX': 23, 'Western': 14, 'Film-Noir': 12, '(no genres listed)': 4}
-----------
{'Drama': 441, 'Comedy': 385, 'Thriller': 204, 'Action': 180, 'Romance': 159, 'Adventure': 140, 'Crime': 136, 'Horror': 109, 'Sci-Fi': 107, 'Children': 67, 'Fantasy': 66, 'Mystery': 64, 'Documentary': 61, 'Animation': 51, 'War': 35, 'Musical': 33, 'Western': 16, 'IMAX': 10, 'Film-Noir': 7, '(no genres listed)': 1}


## Адаптация собственного алгоритма

Адаптируйте ваш алгоритм из первой лабораторной для совместимости с библиотекой согласно документации: https://surprise.readthedocs.io/en/stable/building_custom_algo.html

Сравните настроенные ранее модели со своим решением с помощью кроссвалидации.

In [38]:
# Сам датасет
ratings = pd.read_csv('./ml-latest-small/ratings.csv', delimiter=',')
# Оставляем в таблице только нужные столбцы
ratings = ratings.loc[:, ["userId", "movieId", "rating"]]

# Строка в таблице в датасета это id пользователя, id фильма, рейтинг
# На id пользователей нам плевать, а фильмы хочется смотреть по названиям,
# поэтому загружаем табличку сопоставления названий фильмов их id
movies = pd.read_csv('./ml-latest-small/movies.csv', delimiter=',')

In [267]:
from surprise import AlgoBase, Dataset
from surprise.model_selection import cross_validate


class MyOwnAlgorithm(AlgoBase):
    def __init__(self):
        # Always call base method before doing anything.
        AlgoBase.__init__(self)
        
        self.all_cosine_similarities = {}
        self.mid = {}
        self.filmid = {}
        self.mark = {}
        self.all = {}
        
    def calculate_cosine_similarity(self, u, v):
        dot_product = np.dot(u, v)
        norm_u = np.linalg.norm(u) #np.sqrt(u.dot(u))
        norm_v = np.linalg.norm(v) #np.sqrt(v.dot(v))
        return dot_product / (norm_u * norm_v)
    
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        
        for user, movie_rating in trainset.ur.items(): # user - пользователь, movie_rating - список(фильм, оценка)
            self.filmid[user] = []
            self.mark[user] = []
            all_user = {}
            
            # проходимся по всем оценкам конкретного пользователя
            for i in range (len(trainset.ur[user])):
                self.filmid[user].append(movie_rating[i][0])
                self.mark[user].append(movie_rating[i][1])
                all_user[movie_rating[i][0]] = movie_rating[i][1]
            self.all[user] = all_user

        for user_u, movie_rating_u in trainset.ur.items(): # user_u = int(user_id), movies_u = tuple(movie_id, rating)
            cosine_similarity = {} # косинусное сходство конкретного пользователя со всеми остальными
            
            sum_u = 0
            s_downk = 0
            s_downl = 0
            for k in range (len(trainset.ur[user_u])):
                sum_u += trainset.ur[user_u][k][1]
            s_downk = sum(i*i for i in self.mark[user_u])
            
            movies_u = {}
            for elem in trainset.ur[user_u]:
                movies_u[elem[0]] = elem[1]
            
            for user_v, movie_rating_v in trainset.ur.items(): # user_v = int(user_id), movies_v = tuple(movie_id, rating)
                vec_u = np.array([])
                vec_v = np.array([])
                if (user_v != user_u):
                    s_up = 0
                    s_downl = sum(i*i for i in self.mark[user_v])
                    a = list(set(self.filmid[user_u]) & set(self.filmid[user_v]))
                    for j in a:
                        s_up += self.mark[user_u][self.filmid[user_u].index(j)]*self.mark[user_v][self.filmid[user_v].index(j)]
                    for movies_v in movie_rating_v:
                        #print(movie_rating_v)
                        if movies_v[0] in movies_u:
                            vec_u = np.append(vec_u, movies_u[movies_v[0]])
                            vec_v = np.append(vec_v, movies_v[1])
                            #print(vec_u)
                            #print(vec_v)
                            #print()
                            
                            if (len(vec_u) > 1):
                                cosine_similarity[user_v] = self.calculate_cosine_similarity(vec_u, vec_v)
                            else:
                                cosine_similarity[user_v] = 0.0
                            
                self.all_cosine_similarities[user_u] = cosine_similarity
                self.mid[user_u] = sum_u / len(trainset.ur[user_u])
        #print(self.all_cosine_similarities)  
        return self
        
    def estimate(self, u, i):
        # может сложиться ситуация, когда пользователь ни разу не попал в обучающую выборку
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            #raise PredictionImpossible("User and/or item is unknown.")
            return 3.5 # тогда вернем среднее
        r_up = 0
        r_down = 0
        for v, _ in trainset.ur.items(): # здесь trainset берется извне
            if (v != u and i in self.all[v]):
                try:
                    r_up += self.all_cosine_similarities[u][v] * (self.all[v][i] - self.mid[v])
                    r_down += abs(self.all_cosine_similarities[u][v])
                except Exception as e:
                    s = str(e)
                    return 3.50
        if r_down == 0:
            bsl = 0.0
        else:
            bsl = self.mid[v] + r_up / r_down
        return bsl


In [268]:
algo = MyOwnAlgorithm()

cross_validate(algo, data, measures=["mse", "rmse", "mae"], cv=5, verbose=True)

Evaluating MSE, RMSE, MAE of algorithm MyOwnAlgorithm on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MSE (testset)     0.9853  0.9604  0.9645  0.9736  0.9738  0.9715  0.0086  
RMSE (testset)    0.9926  0.9800  0.9821  0.9867  0.9868  0.9856  0.0044  
MAE (testset)     0.7722  0.7699  0.7751  0.7760  0.7716  0.7730  0.0023  
Fit time          74.77   75.64   75.86   76.07   75.78   75.62   0.45    
Test time         1.74    1.81    1.80    1.79    1.95    1.82    0.07    


{'test_mse': array([0.98531522, 0.96041853, 0.96445775, 0.97355105, 0.97375212]),
 'test_rmse': array([0.99263046, 0.98000945, 0.9820681 , 0.98668691, 0.98678879]),
 'test_mae': array([0.77221849, 0.76992476, 0.77514373, 0.77604667, 0.77158846]),
 'fit_time': (74.76871132850647,
  75.63512301445007,
  75.85923218727112,
  76.06636071205139,
  75.77789235115051),
 'test_time': (1.7353603839874268,
  1.8051683902740479,
  1.8017966747283936,
  1.789731502532959,
  1.9487874507904053)}

# Вывод
Модель на косинуснум сходстве приблизилась по качеству к SVD модели без подбора гиперпараметров, но неотрицательное матричное разложение немного лучше. Результат полученной модели можно еще улучшать, если считать неблизкими пользователей, у которых мало совпавших фильмов, и подбирать коэффициенты